<a href="https://colab.research.google.com/github/vg2425/W4111-Intro-to-DB-S22/blob/main/MC_sim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* define index $bn
* create a pandas table for each case (3)
* create function to run each case
* create function to run x value based on c and d values
* create function to run final outcome


In [117]:
import pandas as pd
import numpy as np

In [118]:
index_dollars_1 = ["$30.0bn", "$32.5bn", "$35.0bn", "$37.5bn", "$40.0bn", "$42.5bn"]
index_dollars_2 = ["$32.5bn", "$35.0bn", "$37.5bn", "$40.0bn", "$42.5bn"]
quartile_index = ["Low", "Delta", "High", "Average", "Median"]
case_base = pd.DataFrame(index = index_dollars_1, columns = quartile_index)
case_1 = pd.DataFrame(index = index_dollars_1, columns = quartile_index)
case_2 = pd.DataFrame(index = index_dollars_2, columns = quartile_index)

In [119]:
test = np.array([325]*iterations)
c_values = np.array([32.5*10]*iterations)
np.minimum(test, c_values)

array([325., 325., 325., 325., 325., 325., 325., 325., 325., 325.])

In [120]:
def row_analysis(crane_value, case_toggle, iterations):
  rng = np.random.default_rng(10)
  iterations = 10
  #d ranges
  d_low = 255
  d_high = 275
  #generate c values
  c_values = np.array([crane_value*10]*iterations)
  #generate d values
  d_values = rng.integers(d_low, d_high, iterations, endpoint = True)
  #generate x values based on c and d
  x_low = d_values
  x_high = c_values
  if case_toggle == 1:
    x_low = np.array([275]*iterations)
    x_high = np.minimum(c_values, np.array([325]*iterations))
  elif case_toggle == 2:
    x_low = np.array([325]*iterations)
    x_high = np.minimum(c_values, np.array([375]*iterations))

  x_values = rng.integers(x_low, x_high)

  #divide values by 10 to get 10 billion units
  c_values = c_values/10
  d_values = d_values/10
  x_values = x_values/10

  #dataframe that combines all bids
  bids = pd.DataFrame([d_values, c_values, x_values], index = ["d", "c", "x"])
  bids = bids.T

  #calculate absolute differences between dx, cx, dc
  abs_dx = np.absolute(d_values - x_values)
  abs_cx = np.absolute(c_values - x_values)
  abs_dc = np.absolute(d_values - c_values)

  #table of absolute value differences
  absolute_vals = pd.DataFrame([abs_dx, abs_cx, abs_dc], index = [1, 2, 3]) #1 = abs dx, 2 = abs cx, 3 = abs dc
  absolute_vals = absolute_vals.T

  #calculate which difference is the smallest between three, get column ID for each row
  minvalue_series = absolute_vals.idxmin(axis = 1)
  bids["mincol"] = minvalue_series #add column index of min value to table with bids

  #get outcomes
  bids_1 = bids[bids["mincol"] == 1].assign(outcome = (bids["d"]+bids["x"])/2)
  bids_2 = bids[bids["mincol"] == 2].assign(outcome = (bids["c"]+bids["x"])/2)
  bids_3 = bids[bids["mincol"] == 3].assign(outcome = (bids["d"]+bids["c"])/2)

  bids_1.dropna(how = "any", inplace = True)
  bids_2.dropna(how = "any", inplace = True)
  bids_3.dropna(how = "any", inplace = True)

  outcome = pd.concat([bids_1, bids_2, bids_3], axis=0)
  outcome["floor"] = 27.5
  outcome["final"] = outcome[["outcome", "floor"]].max(axis=1)

  analysis = outcome["final"].describe()
  return analysis[4], analysis[6]-analysis[4], analysis[6], analysis[1], analysis[5]

In [121]:
def run_base():
  case_base[0:1] = row_analysis(30, 0, 10000)
  case_base[1:2] = row_analysis(32.5, 0, 10000)
  case_base[2:3] = row_analysis(35, 0, 10000)
  case_base[3:4] = row_analysis(37.5, 0, 10000)
  case_base[4:5] = row_analysis(40, 0, 10000)
  case_base[5:6] = row_analysis(42.5, 0, 10000)

def run_1():
  case_1[0:1] = row_analysis(30, 1, 10000)
  case_1[1:2] = row_analysis(32.5, 1, 10000)
  case_1[2:3] = row_analysis(35, 1, 10000)
  case_1[3:4] = row_analysis(37.5, 1, 10000)
  case_1[4:5] = row_analysis(40, 1, 10000)
  case_1[5:6] = row_analysis(42.5, 1, 10000)

def run_2():
  case_2[0:1] = [32.5, 0, 32.5, 32.5, 32.5]
  case_2[1:2] = row_analysis(35, 2, 10000)
  case_2[2:3] = row_analysis(37.5, 2, 10000)
  case_2[3:4] = row_analysis(40, 2, 10000)
  case_2[4:5] = row_analysis(42.5, 2, 10000)

In [122]:
run_base()
run_1()
run_2()

In [123]:
case_base

,Low,Delta,High,Average,Median
$30.0bn,27.5,1.8375,29.3375,28.36,27.65
$32.5bn,27.5,3.8875,31.3875,29.3,27.975
$35.0bn,27.7625,5.675,33.4375,30.32,28.325
$37.5bn,28.175,7.3125,35.4875,31.38,28.75
$40.0bn,28.425,9.1625,37.5875,32.46,29.275
$42.5bn,28.7125,10.925,39.6375,33.53,29.775


In [124]:
case_1

,Low,Delta,High,Average,Median
$30.0bn,27.6875,1.8625,29.55,28.765,29.25
$32.5bn,27.925,3.675,31.6,30.075,31.025
$35.0bn,27.7125,4.1625,31.875,29.525,28.05
$37.5bn,27.7125,0.875,28.5875,28.805,28.05
$40.0bn,27.7125,0.875,28.5875,28.255,28.05
$42.5bn,27.7125,0.875,28.5875,28.255,28.05


In [125]:
case_2

,Low,Delta,High,Average,Median
$32.5bn,32.5,0,32.5,32.5,32.5
$35.0bn,33.9875,0.5625,34.55,34.3,34.25
$37.5bn,35.5125,1.0875,36.6,36.115,36.025
$40.0bn,32.0375,5.8125,37.85,35.42,37.275
$42.5bn,30.425,8.675,39.1,35.525,38.525


In [126]:
case_base.to_excel("base.xlsx")
case_1.to_excel("case1.xlsx")
case_2.to_excel("case2.xlsx")